In [1]:
from texascovid19 import constants
import requests
import pandas as pd
from datetime import date

Fetch each day's data.  The data is "current as of 8 p.m. the day before reporting."

In [2]:
r = requests.get(constants.DSHS_XHR_URL, verify=False)
r.raise_for_status()

/Users/dan/code/texascovid19/venv/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'services5.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [4]:
county_data = r.json()['features']

In [5]:
norm_county_data = []
for county in county_data:
        norm_county_data.append({
            'County of Residence': county['attributes']['County'],
            'Deaths': county['attributes']['Deaths'] or 0,
            'Number of Cases': county['attributes']['Count_']
        })

In [6]:
df = pd.DataFrame(data=norm_county_data)

In [7]:
df

,County of Residence,Deaths,Number of Cases
0,Dallas,5,169
1,Harris,1,134
2,Travis,0,98
3,Tarrant,1,71
4,Bexar,1,69
...,...,...,...
77,Tom Green,0,1
78,Upshur,0,1
79,Van Zandt,0,1
80,Walker,0,1


In [8]:
df.set_index("County of Residence", inplace=True)

In [10]:
today = date.today().isoformat()

In [11]:
df.to_csv(f"{constants.DATA_PATH}/{today}.csv", header=True, index=True)

Append today's data to timeseries

In [12]:
df_timeseries = pd.read_csv(f"{constants.TIMESERIES_CSV}").set_index("County of Residence")

In [13]:
df.rename(columns={"Number of Cases": today}, inplace=True)

In [14]:
df.drop("Deaths", axis=1, inplace=True)

In [15]:
if today in df_timeseries.columns:
    df_timeseries.drop(columns=today, inplace=True)

In [16]:
df_timeseries = pd.concat([df_timeseries, df], axis=1).rename_axis('County of Residence')

In [17]:
df_timeseries = df_timeseries.fillna(0).convert_dtypes(convert_integer=True)

In [18]:
df_timeseries.to_csv(constants.TIMESERIES_CSV, header=True, index=True)